# Creating Delta Quantity Data
Some of the figures and tables in the article are related to the "delta_quantity_under_1b.npz" file. In this notebook, the codes required to reproduce this file are written.

In [1]:
from pathlib import Path
from fractions import Fraction
import numpy as np
from numba import jit, uint64

First of all, we need some prime number; For this purpuse I defined `eratosthenes_sieve` as follow:

In [3]:
@jit(uint64[:](uint64), nopython=True, cache=True)
def eratosthenes_sieve(stop: int):
    numbers = np.arange(2, stop, dtype=np.uint64)
    for n in numbers:
        if n != 0:
            i = 2
            while i*n-2 < stop-1:          
                numbers[i*n-2] = 0
                i += 1
        else:
            continue
    primes = numbers[numbers != 0]
    return primes

Set `maxnumber` to a any number you want; Pass this variable to `eratosthenes_sieve` function as `stop` argument and this function will produce all primes up to `maxnumber`.

In [4]:
maxnumber = 10000
primes = eratosthenes_sieve(maxnumber)
print(f'Number of produced primes: {len(primes)}')

Number of produced primes: 1229


By using following function we obtain the quantity of deltas:

In [5]:
def count_deltas(primes, order):
    last_index = primes.size - 1
    max_order = last_index//2
    if 2*order > last_index:
        raise ValueError(f'Maximum allowable order is {max_order}')
    deltas_set = set()
    deltas_count = np.zeros((primes.size-2*order))
    i = 0
    while i+2*order <= last_index:
        p1, p2, p3 = primes[i], primes[i+order], primes[i+2*order]
        delta = str(Fraction(p3.item()-p2.item(), p2.item()-p1.item()))
        deltas_set.add(delta)
        deltas_count[i] = len(deltas_set)
        i += 1
    return deltas_count

To get quantity of a specific order of delta, use `count_deltas(primes, order)`. This function returns an array which each index of it corresponds to a prime number, and the value at that index is the quantity of specified delta up to that prime.

In [11]:
delta_quantity_order_1 = count_deltas(primes, 1)
delta_quantity_order_1

array([ 1.,  2.,  2., ..., 85., 85., 85.])

## Validate `delta_quantity_under_1b.npz`

Now lets compare results from this function to the data in `'delta_quantity_under_1b.npz'` First lets examine this file. This file is a collection of arrays. Each array contains delta quantity for a specific order.

In [13]:
delta_quantity_path = Path('delta_quantity_under_1b.npz')
try:
    delta_quantity = np.load(delta_quantity_path)
except FileNotFoundError as error:
        print(error)
        in_colab = 'google.colab' in sys.modules
        if in_colab:
            print('running on colab, downloading data to session...')
            !mkdir data
            # Download delta_quantity_under_1b.npz
            !wget https://bit.ly/3fZDjtY -O data/delta_quantity_under_1b.npz
            delta_quantity = np.load(delta_quantity_path)
        else:
            print('Please download required data from paper repo')
list(delta_quantity.keys())

['order_1', 'order_10', 'order_100', 'order_1000']

First generate all primes up to 1 bilion:

In [ ]:
maxnumber = 1000000000
primes = eratosthenes_sieve(maxnumber)
print(f'Number of produced primes: {len(primes)}')

Now produce delta quantity arrays; Specify order and check for validity of `delta_quantity_under_1b.npz`

In [ ]:
order = 1 # Order should be 1, 10 or 1000
delta_quantity_in_file = delta_quantity[f'order_{order}']
produced_delta_quantity = count_deltas(primes, 1)
if delta_quantity_in_file == produced_delta_quantity:
    print(f'delta_quantity_under_1b.npz for order {order} is valid')